## Aggregating Data

Let us understand how to aggregate the data.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [24]:
val username = System.getProperty("user.name")

username = itv002461


lastException: Throwable = null


itv002461

In [25]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Basic Transformations").
    master("yarn").
    getOrCreate

username = itv002461
spark = org.apache.spark.sql.SparkSession@430240f3


org.apache.spark.sql.SparkSession@430240f3

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* We can perform global aggregations as well as aggregations by key.
* Global Aggregations
  * Get total number of orders.
  * Get revenue for a given order id.
  * Get number of records with order_status either COMPLETED or CLOSED.
* Aggregations by key - using `GROUP BY`
  * Get number of orders by date or status.
  * Get revenue for each order_id.
  * Get daily product revenue (using order date and product id as keys).
* We can also use `HAVING` clause to apply filtering on top of aggregated data.
  * Get daily product revenue where revenue is greater than $500 (using order date and product id as keys).
* Rules while using `GROUP BY`.
  * We can have the columns which are specified as part of `GROUP BY` in `SELECT` clause.
  * On top of those, we can have derived columns using aggregate functions.
  * We cannot have any other columns that are not used as part of `GROUP BY` on derived column using non aggregate functions.
  * We will not be able to use aggregate functions or aliases used in the select clause as part of the where clause.
  * If we want to filter based on aggregated results, then we can leverage `HAVING` on top of `GROUP BY` (specifying `WHERE` is not an option)
* Typical query execution - FROM -> WHERE -> GROUP BY -> SELECT

In [26]:
%%sql 
use itv002461_retail

++
||
++
++



In [27]:
%%sql

SELECT count(order_id) FROM orders

+---------------+
|count(order_id)|
+---------------+
|          68883|
+---------------+



In [28]:
%%sql

SELECT count(DISTINCT order_date) FROM orders

+--------------------------+
|count(DISTINCT order_date)|
+--------------------------+
|                       364|
+--------------------------+



In [29]:
%%sql

SELECT round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items 
WHERE order_item_order_id = 2

+-------------+
|order_revenue|
+-------------+
|       579.98|
+-------------+



In [30]:
%%sql

SELECT count(1) 
FROM orders
WHERE order_status IN ('COMPLETE', 'CLOSED')

+--------+
|count(1)|
+--------+
|   30455|
+--------+



In [31]:
%%sql

SELECT order_date,
    count(1)
FROM orders
GROUP BY order_date

+--------------------+--------+
|          order_date|count(1)|
+--------------------+--------+
|2013-08-13 00:00:...|      73|
|2014-03-19 00:00:...|     130|
|2014-04-26 00:00:...|     251|
|2013-10-12 00:00:...|     162|
|2013-11-15 00:00:...|     135|
|2013-09-16 00:00:...|     121|
|2013-09-20 00:00:...|     139|
|2013-12-31 00:00:...|     266|
|2014-06-15 00:00:...|     128|
|2013-09-06 00:00:...|     276|
+--------------------+--------+
only showing top 10 rows



In [32]:
%%sql

SELECT order_status,
    count(1) AS status_count
FROM orders
GROUP BY order_status

+---------------+------------+
|   order_status|status_count|
+---------------+------------+
|PENDING_PAYMENT|       15030|
|       COMPLETE|       22899|
|        ON_HOLD|        3798|
| PAYMENT_REVIEW|         729|
|     PROCESSING|        8275|
|         CLOSED|        7556|
|SUSPECTED_FRAUD|        1558|
|        PENDING|        7610|
|       CANCELED|        1428|
+---------------+------------+



In [33]:
%%sql

SELECT order_item_order_id,
    round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items
GROUP BY order_item_order_id LIMIT 10

+-------------------+-------------+
|order_item_order_id|order_revenue|
+-------------------+-------------+
|                148|       479.99|
|                463|       829.92|
|                471|       169.98|
|                496|       441.95|
|               1088|       249.97|
|               1580|       299.95|
|               1591|       439.86|
|               1645|      1509.79|
|               2366|       299.97|
|               2659|       724.91|
+-------------------+-------------+



In [34]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
LIMIT 10

+--------------------+-----...


+--------------------+---------------------+-------+
|          order_date|order_item_product_id|revenue|
+--------------------+---------------------+-------+
|2014-03-11 00:00:...|                  135|  132.0|
|2014-03-26 00:00:...|                  897|  24.99|
|2014-04-02 00:00:...|                 1073|6399.68|
|2014-04-04 00:00:...|                  565|  210.0|
|2014-05-07 00:00:...|                  642|  120.0|
|2014-05-18 00:00:...|                  835| 127.96|
|2014-05-28 00:00:...|                  191|5399.46|
|2014-06-02 00:00:...|                  627|  799.8|
|2014-06-05 00:00:...|                  627|1079.73|
|2014-06-18 00:00:...|                  273| 111.96|
+--------------------+---------------------+-------+



In [35]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
    AND revenue >= 500
GROUP BY o.order_date,
    oi.order_item_product_id
LIMIT 10

            :  +- SubqueryAlia...


Magic sql failed to execute with error: 
cannot resolve '`revenue`' given input columns: [oi.order_item_quantity, oi.order_item_id, o.order_date, oi.order_item_product_price, o.order_customer_id, oi.order_item_subtotal, oi.order_item_order_id, o.order_id, o.order_status, oi.order_item_product_id]; line 1 pos 236;
'GlobalLimit 10
+- 'LocalLimit 10
   +- 'Aggregate ['o.order_date, 'oi.order_item_product_id], ['o.order_date, 'oi.order_item_product_id, 'round('sum('oi.order_item_subtotal), 2) AS revenue#494]
      +- 'Filter (order_status#498 IN (COMPLETE,CLOSED) && ('revenue >= 500))
         +- Join Inner, (order_id#495 = order_item_order_id#500)
            :- SubqueryAlias `o`
            :  +- SubqueryAlias `itv002461_retail`.`orders`
            :     +- HiveTableRelation `itv002461_retail`.`orders`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [order_id#495, order_date#496, order_customer_id#497, order_status#498]
            +- SubqueryAlias `oi`
               +- SubqueryAl

In [36]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
HAVING revenue >= 500
LIMIT 10

+--------------------+-----...


+--------------------+---------------------+-------+
|          order_date|order_item_product_id|revenue|
+--------------------+---------------------+-------+
|2014-04-09 00:00:...|                  191|6599.34|
|2014-05-29 00:00:...|                 1014|2698.92|
|2014-06-01 00:00:...|                 1073|3199.84|
|2014-06-30 00:00:...|                  502| 4000.0|
|2013-08-12 00:00:...|                  627| 3199.2|
|2013-09-04 00:00:...|                  957|3599.76|
|2013-10-19 00:00:...|                 1004|5599.72|
|2013-11-06 00:00:...|                  502| 3800.0|
|2014-02-13 00:00:...|                  403|4159.68|
|2014-03-06 00:00:...|                  191| 4999.5|
+--------------------+---------------------+-------+



* Using Spark SQL with Python or Scala

In [37]:
spark.sql("SELECT count(order_id) FROM orders").show()

+---------------+
|count(order_id)|
+---------------+
|          68883|
+---------------+



In [38]:
spark.sql("SELECT count(DISTINCT order_date) FROM orders").show()

+--------------------------+
|count(DISTINCT order_date)|
+--------------------------+
|                       364|
+--------------------------+



In [39]:
spark.sql("""
SELECT round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items 
WHERE order_item_order_id = 2
""").show()

+-------------+
|order_revenue|
+-------------+
|       579.98|
+-------------+



In [40]:
spark.sql("""
SELECT count(1) 
FROM orders
WHERE order_status IN ('COMPLETE', 'CLOSED')
""").show()

+--------+
|count(1)|
+--------+
|   30455|
+--------+



In [41]:
spark.sql("""
SELECT order_date,
    count(1)
FROM orders
GROUP BY order_date
""").show()

+--------------------+--------+
|          order_date|count(1)|
+--------------------+--------+
|2013-08-13 00:00:...|      73|
|2013-10-12 00:00:...|     162|
|2013-11-15 00:00:...|     135|
|2014-03-19 00:00:...|     130|
|2014-04-26 00:00:...|     251|
|2013-09-16 00:00:...|     121|
|2013-09-20 00:00:...|     139|
|2013-12-31 00:00:...|     266|
|2014-06-15 00:00:...|     128|
|2013-09-06 00:00:...|     276|
|2013-12-24 00:00:...|     170|
|2014-06-07 00:00:...|     191|
|2014-01-07 00:00:...|     163|
|2013-10-14 00:00:...|     139|
|2013-11-11 00:00:...|     246|
|2014-01-27 00:00:...|     163|
|2014-01-29 00:00:...|     158|
|2014-02-14 00:00:...|     174|
|2014-04-15 00:00:...|     180|
|2014-04-22 00:00:...|     144|
+--------------------+--------+
only showing top 20 rows



In [42]:
spark.sql("""
SELECT order_status,
    count(1) AS status_count
FROM orders
GROUP BY order_status
""").show()

+---------------+------------+
|   order_status|status_count|
+---------------+------------+
|PENDING_PAYMENT|       15030|
|       COMPLETE|       22899|
|        ON_HOLD|        3798|
| PAYMENT_REVIEW|         729|
|     PROCESSING|        8275|
|         CLOSED|        7556|
|SUSPECTED_FRAUD|        1558|
|        PENDING|        7610|
|       CANCELED|        1428|
+---------------+------------+



In [43]:
spark.sql("""
SELECT order_item_order_id,
    round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items
GROUP BY order_item_order_id
""").show()

+-------------------+-------------+
|order_item_order_id|order_revenue|
+-------------------+-------------+
|              35351|       629.93|
|              35361|       614.88|
|              35689|       799.88|
|              35694|       889.94|
|              35820|       949.96|
|              35912|       799.96|
|              35947|       519.96|
|              35982|      1201.82|
|              36131|       554.95|
|              36224|       729.96|
|              36538|       299.98|
|              37111|       479.92|
|              37146|       329.98|
|              37251|       689.87|
|              37307|       849.94|
|              37489|       329.94|
|              38153|       761.91|
|              38220|       689.89|
|              38311|        99.98|
|              38422|      1249.91|
+-------------------+-------------+
only showing top 20 rows



In [44]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
""").show()

+--------------------+---------------------+-------+
|          order_date|order_item_product_id|revenue|
+--------------------+---------------------+-------+
|2014-03-28 00:00:...|                  793|  59.96|
|2014-04-09 00:00:...|                  191|6599.34|
|2014-04-10 00:00:...|                  775|   9.99|
|2014-04-15 00:00:...|                  116| 404.91|
|2014-05-03 00:00:...|                  172|  120.0|
|2014-05-24 00:00:...|                  249| 164.91|
|2014-05-29 00:00:...|                 1014|2698.92|
|2014-06-01 00:00:...|                 1073|3199.84|
|2014-06-06 00:00:...|                  810|  39.98|
|2014-06-08 00:00:...|                  792|  89.94|
|2014-06-30 00:00:...|                  502| 4000.0|
|2014-07-01 00:00:...|                  926|  31.98|
|2014-07-02 00:00:...|                  793|  14.99|
|2013-08-12 00:00:...|                  627| 3199.2|
|2013-09-04 00:00:...|                  957|3599.76|
|2013-10-19 00:00:...|                 1004|55

In [45]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
    AND revenue >= 500
GROUP BY o.order_date,
    oi.order_item_product_id
""").show()

org.apache.spark.sql.AnalysisException: cannot resolve '`revenue`' given input columns: [oi.order_item_product_id, o.order_customer_id, oi.order_item_product_price, o.order_id, oi.order_item_id, oi.order_item_subtotal, o.order_date, oi.order_item_order_id, oi.order_item_quantity, o.order_status]; line 8 pos 8;
'Aggregate ['o.order_date, 'oi.order_item_product_id], ['o.order_date, 'oi.order_item_product_id, 'round('sum('oi.order_item_subtotal), 2) AS revenue#672]
+- 'Filter (order_status#676 IN (COMPLETE,CLOSED) && ('revenue >= 500))
   +- Join Inner, (order_id#673 = order_item_order_id#678)
      :- SubqueryAlias `o`
      :  +- SubqueryAlias `itv002461_retail`.`orders`
      :     +- HiveTableRelation `itv002461_retail`.`orders`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [order_id#673, order_date#674, order_customer_id#675, order_status#676]
      +- SubqueryAlias `oi`
         +- SubqueryAlias `itv002461_retail`.`order_items`
            +- HiveTableRelation `itv002461_reta

In [46]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
HAVING revenue >= 500
""").show()

+--------------------+---------------------+-------+
|          order_date|order_item_product_id|revenue|
+--------------------+---------------------+-------+
|2014-04-09 00:00:...|                  191|6599.34|
|2014-05-29 00:00:...|                 1014|2698.92|
|2014-06-01 00:00:...|                 1073|3199.84|
|2014-06-30 00:00:...|                  502| 4000.0|
|2013-08-12 00:00:...|                  627| 3199.2|
|2013-09-04 00:00:...|                  957|3599.76|
|2013-10-19 00:00:...|                 1004|5599.72|
|2013-11-06 00:00:...|                  502| 3800.0|
|2014-02-13 00:00:...|                  403|4159.68|
|2013-08-04 00:00:...|                  403|3119.76|
|2013-09-08 00:00:...|                 1073|3199.84|
|2013-09-09 00:00:...|                  957|5399.64|
|2013-09-15 00:00:...|                  627|1039.74|
|2013-09-22 00:00:...|                  365|6058.99|
|2013-10-03 00:00:...|                  565|  700.0|
|2013-11-03 00:00:...|                  957|80

lastException: Throwable = null
